## Question 1

In [47]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("assets/nhl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def nhl_correlation():
    # read the files
    nhl_df = pd.read_csv("assets/nhl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    # select year = 2018 and create a column 'W/L%'
    nhl_df = nhl_df.query('year == 2018')
    nhl_df = nhl_df[nhl_df['W'].str.isnumeric() & nhl_df['L'].str.isnumeric()].astype({
        'W': int, 'L': int}).reset_index(drop=True)
    nhl_df['W/L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    # split teams names in nhl_df
    nhl_df['team'] = nhl_df['team'].str.replace(r'\*+', '', regex=True)
    nhl_df['team'] = nhl_df['team'].str.split(' ').str[-1]

    # split teams names in cities
    team = {'RangersIslandersDevils': 'Rangers Islanders Devils',
            'KingsDucks': 'Kings Ducks'}
    cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]', '', regex=True)
    cities['NHL'] = cities['NHL'].replace(team)
    cities['NHL'] = cities['NHL'].str.split(' ')
    cities = cities.explode('NHL')

    # merge nhl_df and cities
    df = pd.merge(nhl_df, cities, left_on='team', right_on='NHL')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL', 'W/L%']]

    # mean for same cities
    df_mean = df.groupby('Metropolitan area')[['W/L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area')
    df = df[['Metropolitan area',
             'Population (2016 est.)[8]', 'NHL', 'W/L%_y']].rename(columns={'W/L%_y': 'W/L%'})
    df = df.drop_duplicates('Metropolitan area').reset_index(drop=True)

    population_by_region = df['Population (2016 est.)[8]'].astype(
        'int64')  # pass in metropolitan area population from cities
    # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df['W/L%']

    assert len(population_by_region) == len(
        win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(
        population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    corr = stats.pearsonr(population_by_region, win_loss_by_region)

    return corr[0]

    raise NotImplementedError()


In [48]:
nhl_correlation()


0.012486162921209892

### Solution

In [ ]:
nhl_df = nhl_df.query('year == 2018')
nhl_df = nhl_df[nhl_df['W'].str.isnumeric() & nhl_df['L'].str.isnumeric()].astype({
    'W': int, 'L': int}).reset_index(drop=True)
nhl_df['W/L'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])
nhl_df


In [ ]:
# retirando o asterisco
nhl_df['team'] = nhl_df['team'].str.replace(r'\*+', '')

# separando os nomes dos times e criando uma nova coluna df com nome dos times
nhl_df['team_sep'] = nhl_df['team'].str.split(' ').str[-1]

nhl_df


In [ ]:
# separando os nomes dos times no cities
teams = {'RangersIslandersDevils': 'Rangers Islanders Devils',
         'KingsDucks': 'Kings Ducks'}
cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]', '')
cities['NHL'] = cities['NHL'].replace(teams)
cities['NHL'] = cities['NHL'].str.split(' ')
cities = cities.explode('NHL')
cities


In [ ]:
# fazendo o merge entre os dataframes
df = pd.merge(nhl_df, cities, left_on='team_sep', right_on='NHL')


In [ ]:
df = df[['Metropolitan area',
         'Population (2016 est.)[8]', 'NHL', 'W', 'L', 'W/L']]
df


In [ ]:
# definindo as médias
df_mean = df.groupby(['Metropolitan area'])[['W/L']].mean()


In [ ]:
# merge entre o df e as médias
df_new = pd.merge(df, df_mean, on='Metropolitan area')
df_new = df_new[['Metropolitan area',
                 'Population (2016 est.)[8]', 'NHL', 'W/L_y']].rename(columns={'W/L_y': 'W/L%'})
df_new = df_new.drop_duplicates('Metropolitan area').reset_index(drop=True)
df_new


In [ ]:
df_new.info()


In [ ]:
population_by_region = df_new['Population (2016 est.)[8]'].astype(
    {'Population (2016 est.)[8]': 'Int64'})
win_loss_by_region = df_new['W/L%']

nhl_cor = stats.pearsonr(population_by_region, win_loss_by_region)
nhl_cor


## Question 2

In [27]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df = pd.read_csv("assets/nba.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def nba_correlation():
    nba_df = pd.read_csv("assets/nba.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    # nba
    nba_df = nba_df.query('year == 2018')
    nba_df['team'] = nba_df['team'].str.replace(
        r'(\*?).\(.+\)', '', regex=True)
    nba_df['team'] = nba_df['team'].str.split(' ').str[-1]

    # cities
    teams = {'KnicksNets': 'Knicks Nets', 'LakersClippers': 'Lakers Clippers'}
    cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]', '', regex=True)
    cities['NBA'] = cities['NBA'].replace(teams)
    cities['NBA'] = cities['NBA'].str.split(' ')
    cities = cities.explode('NBA')

    # merge
    df = pd.merge(nba_df, cities, left_on='team', right_on='NBA')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA', 'W/L%']].astype(
        {'Population (2016 est.)[8]': 'int64', 'W/L%': 'float'})

    # mean and merge
    df_mean = df.groupby('Metropolitan area')[['W/L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area')
    df = df[['Metropolitan area',
             'Population (2016 est.)[8]', 'NBA', 'W/L%_y']].rename(columns={'W/L%_y': 'W/L%'})
    df = df.drop_duplicates('Metropolitan area')

    # pass in metropolitan area population from cities
    population_by_region = df['Population (2016 est.)[8]']
    # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df['W/L%']

    assert len(population_by_region) == len(
        win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(
        population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()


#### Solution

In [28]:
nba_correlation()


C:\Users\jeanm\AppData\Local\Temp\ipykernel_16104\696438505.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_df['team'] = nba_df['team'].str.replace(r'(\*?).\(.+\)', '', regex=True)
C:\Users\jeanm\AppData\Local\Temp\ipykernel_16104\696438505.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_df['team'] = nba_df['team'].str.split(' ').str[-1]


(-0.17636350642182935, 0.36932106185547353)

In [ ]:
nba_df = nba_df.query('year == 2018')
nba_df


In [2]:
# nba
nba_df = nba_df.query('year == 2018')
nba_df['team'] = nba_df['team'].str.replace(r'.\(.+\)', '')
nba_df['team'] = nba_df['team'].str.replace(r'\*+', '')
nba_df['team'] = nba_df['team'].str.split(' ').str[-1]
nba_df


C:\Users\jeanm\AppData\Local\Temp\ipykernel_19160\601805401.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  nba_df['team'] = nba_df['team'].str.replace(r'.\(.+\)','')
C:\Users\jeanm\AppData\Local\Temp\ipykernel_19160\601805401.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  nba_df['team'] = nba_df['team'].str.replace(r'\*+','')


,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA
5,Heat,44,38,0.537,15.0,103.4,102.9,0.15,2018,NBA
6,Bucks,44,38,0.537,15.0,106.5,106.8,-0.45,2018,NBA
7,Wizards,43,39,0.524,16.0,106.6,106.0,0.53,2018,NBA
8,Pistons,39,43,0.47600000000000003,20.0,103.8,103.9,-0.26,2018,NBA
9,Hornets,36,46,0.439,23.0,108.2,108.0,0.07,2018,NBA


In [3]:
# cities
teams = {'KnicksNets': 'Knicks Nets', 'LakersClippers': 'Lakers Clippers'}
cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]', '')
cities['NBA'] = cities['NBA'].replace(teams)
cities['NBA'] = cities['NBA'].str.split(' ')
cities = cities.explode('NBA')
cities


C:\Users\jeanm\AppData\Local\Temp\ipykernel_19160\4062869245.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]','')


,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],Knicks,RangersIslandersDevils[note 3]
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],Nets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,Lakers,KingsDucks
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,Clippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards,Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins


In [13]:
# merge
df = pd.merge(nba_df, cities, left_on='team', right_on='NBA')
df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA', 'W/L%']].astype(
    {'Population (2016 est.)[8]': 'int64', 'W/L%': 'float'})
df


,Metropolitan area,Population (2016 est.)[8],NBA,W/L%
0,Toronto,5928040,Raptors,0.720
1,Boston,4794447,Celtics,0.671
2,Philadelphia,6070500,76ers,0.634
3,Cleveland,2055612,Cavaliers,0.610
4,Indianapolis,2004230,Pacers,0.585
5,Miami–Fort Lauderdale,6066387,Heat,0.537
6,Milwaukee,1572482,Bucks,0.537
7,"Washington, D.C.",6131977,Wizards,0.524
8,Detroit,4297617,Pistons,0.476
9,Charlotte,2474314,Hornets,0.439


In [16]:
# merge
df = pd.merge(nba_df, cities, left_on='team', right_on='NBA')
df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA', 'W/L%']].astype(
    {'Population (2016 est.)[8]': 'int64', 'W/L%': 'float'})
df_mean = df.groupby('Metropolitan area')[['W/L%']].mean()
df = pd.merge(df, df_mean, on='Metropolitan area')
df = df[['Metropolitan area',
         'Population (2016 est.)[8]', 'NBA', 'W/L%_y']].rename(columns={'W/L%_y': 'W/L%'})
df = df.drop_duplicates('Metropolitan area')


28

## Question 3

In [51]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("assets/mlb.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def mlb_correlation():
    mlb_df = pd.read_csv("assets/mlb.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    # mlb
    mlb_df = mlb_df.query('year == 2018')
    mlb_df = mlb_df.replace(
        {'Boston Red Sox': 'RedSox', 'Chicago White Sox': 'WhiteSox'})
    mlb_df['team'] = mlb_df['team'].str.split(' ').str[-1]

    # cities
    teams = {'YankeesMets': 'Yankees Mets', 'DodgersAngels': 'Dodgers Angels',
             'GiantsAthletics': 'Giants Athletics', 'CubsWhite Sox': 'Cubs WhiteSox', 'Red Sox': 'RedSox'}
    cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]', '', regex=True)
    cities['MLB'] = cities['MLB'].replace(teams).str.split(' ')
    cities = cities.explode('MLB')

    # merge
    df = pd.merge(mlb_df, cities, left_on='team', right_on='MLB')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB', 'W-L%']].astype(
        {'Population (2016 est.)[8]': 'int64', 'W-L%': 'float'})

    # mean and merge
    df_mean = df.groupby('Metropolitan area')[['W-L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area').drop_duplicates(
        'Metropolitan area').reset_index(drop=True)
    df = df[['Metropolitan area',
             'Population (2016 est.)[8]', 'MLB', 'W-L%_y']].rename(columns={'W-L%_y': 'W-L%'})

    # pass in metropolitan area population from cities
    population_by_region = df['Population (2016 est.)[8]']
    # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df['W-L%']

    assert len(population_by_region) == len(
        win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(
        population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()


In [52]:
mlb_correlation()


(0.15003737475409495, 0.46442827201123427)

### Solution

In [15]:
mlb_df = mlb_df.query('year == 2018')
mlb_df = mlb_df.replace(
    {'Boston Red Sox': 'RedSox', 'Chicago White Sox': 'WhiteSox'})
mlb_df['team'] = mlb_df['team'].str.split(' ').str[-1]
mlb_df


,team,W,L,W-L%,GB,year,League
0,RedSox,108,54,0.667,--,2018,MLB
1,Yankees,100,62,0.617,8.0,2018,MLB
2,Rays,90,72,0.556,18.0,2018,MLB
3,Jays,73,89,0.451,35.0,2018,MLB
4,Orioles,47,115,0.290,61.0,2018,MLB
5,Indians,91,71,0.562,--,2018,MLB
6,Twins,78,84,0.481,13.0,2018,MLB
7,Tigers,64,98,0.395,27.0,2018,MLB
8,WhiteSox,62,100,0.383,29.0,2018,MLB
9,Royals,58,104,0.358,33.0,2018,MLB


In [16]:
teams = {'YankeesMets': 'Yankees Mets', 'DodgersAngels': 'Dodgers Angels',
         'GiantsAthletics': 'Giants Athletics', 'CubsWhite Sox': 'Cubs WhiteSox', 'Red Sox': 'RedSox'}
cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]', '')
cities['MLB'] = cities['MLB'].replace(teams).str.split(' ')
cities = cities.explode('MLB')
cities


C:\Users\jeanm\AppData\Local\Temp\ipykernel_8308\2152373135.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]', '')


,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],Yankees,KnicksNets,RangersIslandersDevils[note 3]
0,New York City,20153634,GiantsJets[note 1],Mets,KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],Dodgers,LakersClippers,KingsDucks
1,Los Angeles,13310447,RamsChargers[note 4],Angels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],Giants,Warriors,Sharks[note 7]
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs,Bulls[note 9],Blackhawks
3,Chicago,9512999,Bears[note 8],WhiteSox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards[note 11],Capitals


In [21]:
# merge
df = pd.merge(mlb_df, cities, left_on='team', right_on='MLB')
df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB', 'W-L%']].astype(
    {'Population (2016 est.)[8]': 'int64', 'W-L%': 'float'})
df_mean = df.groupby('Metropolitan area')[['W-L%']].mean()
df = pd.merge(df, df_mean, on='Metropolitan area')
df = df[['Metropolitan area',
         'Population (2016 est.)[8]', 'MLB', 'W-L%_y']].rename(columns={'W-L%_y': 'W-L%'})
df = df.drop_duplicates('Metropolitan area').reset_index(drop=True)
df


,Metropolitan area,Population (2016 est.)[8],MLB,W-L%
0,Boston,4794447,RedSox,0.667
1,New York City,20153634,Yankees,0.546
2,Tampa Bay Area,3032171,Rays,0.556
3,Toronto,5928040,Jays,0.451
4,Baltimore,2798886,Orioles,0.290
5,Cleveland,2055612,Indians,0.562
6,Minneapolis–Saint Paul,3551036,Twins,0.481
7,Detroit,4297617,Tigers,0.395
8,Chicago,9512999,WhiteSox,0.483
9,Kansas City,2104509,Royals,0.358


## Question 4

In [45]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def nfl_correlation():
    nfl_df = pd.read_csv("assets/nfl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    # nfl
    nfl_df = nfl_df.query('year == 2018')
    nfl_df['team'] = nfl_df['team'].str.replace(r'(\*|\+)', '', regex=True)
    nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1]

    # cities
    teams = {'GiantsJets': 'Giants Jets',
             'RamsChargers': 'Rams Chargers', '49ersRaiders': '49ers Raiders'}
    cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]', '', regex=True)
    cities['NFL'] = cities['NFL'].replace(teams).str.split(' ')
    cities = cities.explode('NFL')

    # merge
    df = pd.merge(nfl_df, cities, left_on='team', right_on='NFL')
    df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL', 'W-L%']].astype(
        {'Population (2016 est.)[8]': 'int64', 'W-L%': 'float'})
    df_mean = df.groupby('Metropolitan area')[['W-L%']].mean()
    df = pd.merge(df, df_mean, on='Metropolitan area').drop_duplicates(
        'Metropolitan area').reset_index(drop=True).rename(columns={'W-L%_y': 'W-L%'})

    # pass in metropolitan area population from cities
    population_by_region = df['Population (2016 est.)[8]']
    # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    win_loss_by_region = df['W-L%']

    assert len(population_by_region) == len(
        win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(
        population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()


In [46]:
nfl_correlation()


C:\Users\jeanm\AppData\Local\Temp\ipykernel_16104\692959198.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df['team'] = nfl_df['team'].str.replace(r'(\*|\+)', '', regex=True)
C:\Users\jeanm\AppData\Local\Temp\ipykernel_16104\692959198.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1]


(0.00428214143639304, 0.9824114740736553)

### Solution

In [ ]:
cities.head(25)


In [40]:
nfl_df = nfl_df.query('year == 2018')
nfl_df['team'] = nfl_df['team'].str.replace(r'(\*|\+)', '', regex=True)
nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1]
nfl_df


,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,Patriots,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,Jets,2018
5,AFC North,AFC North,NFL,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,AFC North,North,2018
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Ravens,2018
7,1.7,6,NFL,4.3,3.9,360,68,428,5.6,1.3,1,9,.594,Steelers,2018
8,0.6,8,NFL,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7,.469,Browns,2018
9,-3.4,10,NFL,-5.4,0.0,455,-87,368,-3.4,2.0,0,6,.375,Bengals,2018


In [41]:
# cities
teams = {'GiantsJets': 'Giants Jets',
         'RamsChargers': 'Rams Chargers', '49ersRaiders': '49ers Raiders'}
cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]', '', regex=True)
cities['NFL'] = cities['NFL'].replace(teams).str.split(' ')
cities = cities.explode('NFL')
cities


,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants,YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
0,New York City,20153634,Jets,YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,Rams,DodgersAngels,LakersClippers,KingsDucks
1,Los Angeles,13310447,Chargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ers,GiantsAthletics,Warriors,Sharks[note 7]
2,San Francisco Bay Area,6657982,Raiders,GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]


In [42]:
# merge

df = pd.merge(nfl_df, cities, left_on='team', right_on='NFL')
df = df[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL', 'W-L%']].astype(
    {'Population (2016 est.)[8]': 'int64', 'W-L%': 'float'})
df_mean = df.groupby('Metropolitan area')[['W-L%']].mean()
df = pd.merge(df, df_mean, on='Metropolitan area').drop_duplicates(
    'Metropolitan area').reset_index(drop=True).rename(columns={'W-L%_y': 'W-L%'})
df


,Metropolitan area,Population (2016 est.)[8],NFL,W-L%_x,W-L%
0,Boston,4794447,Patriots,0.688,0.6880
1,Miami–Fort Lauderdale,6066387,Dolphins,0.438,0.4380
2,Buffalo,1132804,Bills,0.375,0.3750
3,New York City,20153634,Jets,0.250,0.2815
4,Baltimore,2798886,Ravens,0.625,0.6250
5,Pittsburgh,2342299,Steelers,0.594,0.5940
6,Cleveland,2055612,Browns,0.469,0.4690
7,Cincinnati,2165139,Bengals,0.375,0.3750
8,Houston,6772470,Texans,0.688,0.6880
9,Indianapolis,2004230,Colts,0.625,0.6250


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("assets/mlb.csv")
nhl_df = pd.read_csv("assets/nhl.csv")
nba_df = pd.read_csv("assets/nba.csv")
nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()

    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k: np.nan for k in sports}, index=sports)

    assert abs(p_values.loc["NBA", "NHL"] -
               0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] -
               0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


In [ ]:
# cities
teams = {'RangersIslandersDevils': 'Rangers Islanders Devils',
         'KingsDucks': 'Kings Ducks'}
cities['NHL'] = cities['NHL'].str.replace(r'\[.*\]', '', regex=True)
cities['NHL'] = cities['NHL'].replace(team)
cities['NHL'] = cities['NHL'].str.split(' ')

teams = {'KnicksNets': 'Knicks Nets', 'LakersClippers': 'Lakers Clippers'}
cities['NBA'] = cities['NBA'].str.replace(r'\[.*\]', '', regex=True)
cities['NBA'] = cities['NBA'].replace(teams)
cities['NBA'] = cities['NBA'].str.split(' ')

teams = {'YankeesMets': 'Yankees Mets', 'DodgersAngels': 'Dodgers Angels',
         'GiantsAthletics': 'Giants Athletics', 'CubsWhite Sox': 'Cubs WhiteSox', 'Red Sox': 'RedSox'}
cities['MLB'] = cities['MLB'].str.replace(r'\[.*\]', '', regex=True)
cities['MLB'] = cities['MLB'].replace(teams).str.split(' ')

teams = {'GiantsJets': 'Giants Jets',
         'RamsChargers': 'Rams Chargers', '49ersRaiders': '49ers Raiders'}
cities['NFL'] = cities['NFL'].str.replace(r'\[.*\]', '', regex=True)
cities['NFL'] = cities['NFL'].replace(teams).str.split(' ')

cities['NHL'] = cities['NHL'].str.split(' ')
cities['NBA'] = cities['NBA'].str.split(' ')
cities['MLB'] = cities['MLB'].str.split(' ')
cities['NFL'] = cities['NFL'].str.split(' ')


In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("assets/mlb.csv")
nhl_df = pd.read_csv("assets/nhl.csv")
nba_df = pd.read_csv("assets/nba.csv")
nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

# remmember CubsWithe Sox -> Cubs WitheSox
teams = {'RangersIslandersDevils': 'Rangers Islanders Devils',
         'KingsDucks': 'Kings Ducks', 'KnicksNets': 'Knicks Nets', 'LakersClippers': 'Lakers Clippers', 'YankeesMets': 'Yankees Mets', 'DodgersAngels': 'Dodgers Angels',
         'GiantsAthletics': 'Giants Athletics', 'CubsWhite Sox': 'Cubs WhiteSox', 'Red Sox': 'RedSox', 'GiantsJets': 'Giants Jets',
         'RamsChargers': 'Rams Chargers', '49ersRaiders': '49ers Raiders'}

cities[['NHL', 'NBA', 'MLB', 'NFL']] = cities[['NHL', 'NBA', 'MLB', 'NFL']].replace(r'\[.*\]', '', regex=True)
cities[['NHL', 'NBA', 'MLB', 'NFL']] = cities[['NHL', 'NBA', 'MLB', 'NFL']].replace(teams).apply(lambda x: x.str.split(' '))

for x in ['NHL', 'NBA', 'MLB', 'NFL']:
    cities = cities.explode(x) 




In [ ]:
# NHL
nhl_df = nhl_df.query('year == 2018')
nhl_df = nhl_df[nhl_df['W'].str.isnumeric() & nhl_df['L'].str.isnumeric()].astype({
    'W': int, 'L': int}).reset_index(drop=True)
nhl_df['W/L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

# split teams names in nhl_df
nhl_df['team'] = nhl_df['team'].str.replace(r'\*+', '', regex=True)
nhl_df['team'] = nhl_df['team'].str.split(' ').str[-1]
nhl_df = nhl_df[['team', 'W/L%']].rename(columns={'team': 'team_nhl','W/L%': 'W/L%_nhl'}).astype({'W/L%_nhl':'float'})

# merge NHL
nhl_df = pd.merge(nhl_df, cities, left_on='team_nhl', right_on='NHL')
nhl_df = nhl_df[['Metropolitan area','team_nhl', 'W/L%_nhl']].drop_duplicates(subset='team_nhl').reset_index(drop=True)
nhl_df = nhl_df.groupby('Metropolitan area')[['W/L%_nhl']].mean()
nhl_df


In [5]:
# NHL
nhl_df = nhl_df.query('year == 2018')
nhl_df = nhl_df[nhl_df['W'].str.isnumeric() & nhl_df['L'].str.isnumeric()].astype({
    'W': int, 'L': int}).reset_index(drop=True)
nhl_df['W/L%'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

# split teams names in nhl_df
nhl_df['team'] = nhl_df['team'].str.replace(r'\*+', '', regex=True)
nhl_df['team'] = nhl_df['team'].str.split(' ').str[-1]

# merge NHL
nhl_df = pd.merge(nhl_df, cities, left_on='team', right_on='NHL')
nhl_df = (nhl_df.groupby('Metropolitan area')[['W/L%']]
          .mean()
          .merge(nhl_df, on='Metropolitan area')
          .drop_duplicates('Metropolitan area')
          .reset_index(drop=True)
          .rename(columns={'team': 'team_nhl', 'W/L%_y': 'W/L%_nhl'})
          .astype({'W/L%_nhl':'float'})
          )
nhl_df = nhl_df[['Metropolitan area', 'team_nhl', 'W/L%_nhl']]
nhl_df


,Metropolitan area,team_nhl,W/L%_nhl
0,Boston,Bruins,0.714286
1,Buffalo,Sabres,0.357143
2,Calgary,Flames,0.513889
3,Chicago,Blackhawks,0.458333
4,Columbus,Jackets,0.600000
5,Dallas–Fort Worth,Stars,0.567568
6,Denver,Avalanche,0.589041
7,Detroit,Wings,0.434783
8,Edmonton,Oilers,0.473684
9,Las Vegas,Knights,0.680000


In [6]:
# nba
nba_df = nba_df.query('year == 2018')
nba_df['team'] = nba_df['team'].str.replace(r'(\*?).\(.+\)', '', regex=True)
nba_df['team'] = nba_df['team'].str.split(' ').str[-1]

# merge
nba_df = pd.merge(nba_df, cities, left_on='team', right_on='NBA')
nba_df = (nba_df.groupby('Metropolitan area')[['W/L%']]
          .mean()
          .merge(nba_df, on='Metropolitan area')
          .drop_duplicates('Metropolitan area')
          .reset_index(drop=True)
          .rename(columns={'team':'team_nba', 'W/L%': 'W/L%_nba'})
          .astype({'W/L%_nba':'float'}))
nba_df = nba_df[['Metropolitan area', 'team_nba', 'W/L%_nba']]
nba_df


C:\Users\jeanm\AppData\Local\Temp\ipykernel_12104\675471983.py:9: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  .mean()


,Metropolitan area,team_nba,W/L%_nba
0,Atlanta,Hawks,0.293
1,Boston,Celtics,0.671
2,Charlotte,Hornets,0.439
3,Chicago,Bulls,0.329
4,Cleveland,Cavaliers,0.610
5,Dallas–Fort Worth,Mavericks,0.293
6,Denver,Nuggets,0.561
7,Detroit,Pistons,0.476
8,Houston,Rockets,0.793
9,Indianapolis,Pacers,0.585


In [7]:
# mlb
mlb_df = mlb_df.query('year == 2018')
mlb_df = mlb_df.replace({'Boston Red Sox': 'RedSox', 'Chicago White Sox': 'WhiteSox'})
mlb_df['team'] = mlb_df['team'].str.split(' ').str[-1]

#merge
mlb_df = pd.merge(mlb_df, cities, left_on='team', right_on='MLB')
mlb_df = (mlb_df.groupby('Metropolitan area')[['W-L%']]
          .mean()
          .merge(mlb_df, on='Metropolitan area')
          .drop_duplicates('Metropolitan area')
          .reset_index(drop=True)
          .rename(columns={'team': 'team_mlb', 'W-L%_y': 'W-L%_mlb'})
          .astype({'W-L%_mlb':'float'}))

mlb_df = mlb_df[['Metropolitan area', 'team_mlb', 'W-L%_mlb']]
mlb_df

,Metropolitan area,team_mlb,W-L%_mlb
0,Atlanta,Braves,0.556
1,Baltimore,Orioles,0.290
2,Boston,RedSox,0.667
3,Chicago,WhiteSox,0.383
4,Cincinnati,Reds,0.414
5,Cleveland,Indians,0.562
6,Dallas–Fort Worth,Rangers,0.414
7,Denver,Rockies,0.558
8,Detroit,Tigers,0.395
9,Houston,Astros,0.636


In [ ]:
# mlb
mlb_df = mlb_df.query('year == 2018')
mlb_df = mlb_df.replace({'Boston Red Sox': 'RedSox', 'Chicago White Sox': 'WhiteSox'})
mlb_df['team'] = mlb_df['team'].str.split(' ').str[-1]
mlb_df = mlb_df[['team', 'W-L%']].rename(columns={'team':'team_mlb','W-L%': 'W-L%_mlb'}).astype({'W-L%_mlb':'float'})

# merge MLB
mlb_df = pd.merge(mlb_df, cities, left_on='team_mlb', right_on='MLB')
mlb_df = mlb_df.groupby('Metropolitan area')[['W-L%_mlb']].mean()
mlb_df = mlb_df[['Metropolitan area','team_mlb', 'W-L%_mlb']].drop_duplicates(subset='team_mlb').reset_index(drop=True)
mlb_df



In [1]:
972/13

74.76923076923077

In [8]:
# NFL
nfl_df = nfl_df.query('year == 2018')
nfl_df['team'] = nfl_df['team'].str.replace(r'(\*|\+)', '', regex=True)
nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1]

#merge NFL
nfl_df = pd.merge(nfl_df, cities, left_on='team', right_on='NFL')
nfl_df = (nfl_df.groupby('Metropolitan area')[['W-L%']]
          .mean()
          .merge(nfl_df, on='Metropolitan area')
          .drop_duplicates('Metropolitan area')
          .reset_index(drop=True)
          .rename(columns={'team': 'team_nfl', 'W-L%': 'W-L%_nfl'})
          .astype({'W-L%_nfl': 'float'}))

nfl_df = nfl_df[['Metropolitan area', 'team_nfl', 'W-L%_nfl']]
nfl_df

C:\Users\jeanm\AppData\Local\Temp\ipykernel_12104\3308941077.py:9: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  .mean()


,Metropolitan area,team_nfl,W-L%_nfl
0,Atlanta,Falcons,0.438
1,Baltimore,Ravens,0.625
2,Boston,Patriots,0.688
3,Buffalo,Bills,0.375
4,Charlotte,Panthers,0.438
5,Chicago,Bears,0.750
6,Cincinnati,Bengals,0.375
7,Cleveland,Browns,0.469
8,Dallas–Fort Worth,Cowboys,0.625
9,Denver,Broncos,0.375


In [ ]:
# NFL
nfl_df = nfl_df.query('year == 2018')
nfl_df['team'] = nfl_df['team'].str.replace(r'(\*|\+)', '', regex=True)
nfl_df['team'] = nfl_df['team'].str.split(' ').str[-1]
nfl_df = nfl_df[['team', 'W-L%']].rename(columns={'team':'team_nfl','W-L%': 'W-L%_nfl'})

#merge NFL
nfl_df = pd.merge(nfl_df, cities, left_on='team_nfl', right_on='NFL')
nfl_df = nfl_df.groupby('Metropolitan area')[['W-L%_nfl']].mean()
nfl_df = nfl_df[['Metropolitan area','team_nfl', 'W-L%_nfl']].drop_duplicates(subset='team_nfl').reset_index(drop=True).astype({'W-L%_nfl':'float'})
nfl_df


In [9]:
# merge NHL and NBA
df = pd.merge(nba_df, nhl_df, on='Metropolitan area', how='outer')
df = pd.merge(df, mlb_df, on='Metropolitan area', how='outer')
df = pd.merge(df, nfl_df, on='Metropolitan area', how='outer')
df.sort_values(by='Metropolitan area')

,Metropolitan area,team_nba,W/L%_nba,team_nhl,W/L%_nhl,team_mlb,W-L%_mlb,team_nfl,W-L%_nfl
0,Atlanta,Hawks,0.293,NaN,NaN,Braves,0.556,Falcons,0.438
42,Baltimore,NaN,NaN,NaN,NaN,Orioles,0.290,Ravens,0.625
1,Boston,Celtics,0.671,Bruins,0.714286,RedSox,0.667,Patriots,0.688
28,Buffalo,NaN,NaN,Sabres,0.357143,NaN,NaN,Bills,0.375
29,Calgary,NaN,NaN,Flames,0.513889,NaN,NaN,NaN,NaN
2,Charlotte,Hornets,0.439,NaN,NaN,NaN,NaN,Panthers,0.438
3,Chicago,Bulls,0.329,Blackhawks,0.458333,WhiteSox,0.383,Bears,0.750
43,Cincinnati,NaN,NaN,NaN,NaN,Reds,0.414,Bengals,0.375
4,Cleveland,Cavaliers,0.610,NaN,NaN,Indians,0.562,Browns,0.469
30,Columbus,NaN,NaN,Jackets,0.600000,NaN,NaN,NaN,NaN


In [15]:
df1 = df.dropna()
df1

,Metropolitan area,team_nba,W/L%_nba,team_nhl,W/L%_nhl,team_mlb,W-L%_mlb,team_nfl,W-L%_nfl
1,Boston,Celtics,0.671,Bruins,0.714286,RedSox,0.667,Patriots,0.688
3,Chicago,Bulls,0.329,Blackhawks,0.458333,WhiteSox,0.383,Bears,0.750
5,Dallas–Fort Worth,Mavericks,0.293,Stars,0.567568,Rangers,0.414,Cowboys,0.625
6,Denver,Nuggets,0.561,Avalanche,0.589041,Rockies,0.558,Broncos,0.375
7,Detroit,Pistons,0.476,Wings,0.434783,Tigers,0.395,Lions,0.375
10,Los Angeles,Clippers,0.512,Ducks,0.637681,Angels,0.494,Chargers,0.750
12,Miami–Fort Lauderdale,Heat,0.537,Panthers,0.594595,Marlins,0.391,Dolphins,0.438
14,Minneapolis–Saint Paul,Timberwolves,0.573,Wild,0.633803,Twins,0.481,Vikings,0.531
16,New York City,Knicks,0.354,Devils,0.602740,Yankees,0.617,Jets,0.250
19,Philadelphia,76ers,0.634,Flyers,0.617647,Phillies,0.494,Eagles,0.563


In [31]:
df1 = df1.rename(columns={'W/L%_nba': 'NBA', 'W/L%_nhl':'NHL', 'W-L%_mlb':'MLB', 'W-L%_nfl':'NFL'})
sports = ['NFL', 'NBA', 'NHL', 'MLB']
value = []
for x in sports:
    for y in sports:
        result = stats.ttest_rel(df1[x], df1[y])
        value.append(result.statistic)
print(list(set(value)))

[-0.24852244703578297, -0.3398591583149601, 0.24852244703578297, nan, -0.16085985066871375, 1.966320000853937, 2.928157107851504, 3.464526044761038, 0.3398591583149601, 0.16085985066871375, nan, nan, nan, -3.464526044761038, -2.928157107851504, -1.966320000853937]
